In [1]:
from surprise import accuracy, Reader, Dataset, SVD
from surprise.model_selection import cross_validate


In [2]:
pip install scikit-surprise

Note: you may need to restart the kernel to use updated packages.


In [3]:

import pathlib
DATA_DIR = pathlib.Path().resolve().parent / "data"
print(DATA_DIR)
DATA_DIR.exists()

C:\Users\ronis\Desktop\Recommender\recommender\src\cfehome\data


True

In [4]:

import pandas as pd

In [5]:
dataset = DATA_DIR / 'ratings_small.csv'
dataset.exists()

True

In [6]:
df = pd.read_csv(dataset)
df['rating'].dropna(inplace=True)
df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [7]:
df.rating.max(), df.rating.min()

(5.0, 0.5)

In [8]:
reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)

In [9]:
algo = SVD(verbose=True, n_epochs=20)
cross_validate(algo, data, measures=['RMSE', "MAE"], cv=4, verbose=True)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing

{'test_rmse': array([0.90433769, 0.89944217, 0.88948899, 0.90546803]),
 'test_mae': array([0.69752523, 0.69375506, 0.68516376, 0.69709023]),
 'fit_time': (0.7698469161987305,
  0.7400741577148438,
  0.7305800914764404,
  0.7875473499298096),
 'test_time': (0.12279844284057617,
  0.11751961708068848,
  0.12161040306091309,
  0.16445660591125488)}

In [10]:
trainset = data.build_full_trainset()
algo.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19


In [11]:
testset = trainset.build_testset()
predictions = algo.test(testset)
# RMSE should be low as we are biased
accuracy.rmse(predictions, verbose=True)
# accuracy.mae(predictions, verbose=True)

RMSE: 0.6439


0.6439463155562787

In [12]:
sample_row = df.sample(n=1)
userId = sample_row['userId'].values[0]
movieId = sample_row['movieId'].values[0]
print(userId, movieId)

645 2770


In [13]:
pred = algo.predict(uid=userId, iid=movieId)
pred.est

3.331492359050925

In [14]:
import pickle

In [15]:
algo_data = {"model": algo}
with open('model.pkl', 'wb') as f:
    pickle.dump(algo_data, f)

In [16]:
model_algo= None
with open('model.pkl', 'rb') as f:
    model_data_loaded = pickle.load(f)
    model_algo = model_data_loaded.get('model')

In [24]:
model_algo.predict(uid=userId, iid=movieId).est

3.7601631720995967

In [25]:
sample_rows = df.sample(n=10).to_dict('records')
for row in sample_rows:
    userId = row['userId']
    movieId = row['movieId']
    pred = model_algo.predict(uid=userId, iid=movieId).est
    print(userId, movieId, pred)

574 193 2.288075341958605
311 919 3.4333809768916947
544 4995 4.5351678373632325
386 25 3.2137831930323086
506 673 2.3736920480993757
624 90522 1.9244568030422644
264 2081 4.2894846579700285
582 899 4.404176861040419
294 2581 3.583295497013008
108 110 3.6344053504443097
